In [1]:
import os

In [2]:
# change to the root folder!
%pwd

'g:\\Other computers\\PC Luis\\Personal_projects\\end_to_end_ML_projects\\project_laptop_submer\\end-to-end-deep-learning-project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'g:\\Other computers\\PC Luis\\Personal_projects\\end_to_end_ML_projects\\project_laptop_submer\\end-to-end-deep-learning-project'

In [5]:
#update entity now
# entity is defined here!
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True) #it's a data class!
class DataIngestionConfig: # we just declare what variables and what type of variables can be used from this data class
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            # as an input, CONFIG_FILE_PATH and PARAMS_FILE_PATH are always entered (from the constants folder)
            self, #must always be declared as input
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath) #variable config is defined from config_filepath input variable
        self.params = read_yaml(params_filepath) #variable params is defined

        create_directories([self.config.artifacts_root]) #because it requires a list

    def get_data_ingestion_config(self) -> DataIngestionConfig: # so the return must be as DataIngestionConfig data class
        config = self.config.data_ingestion #access the data_ingestion section of the config.yaml

        create_directories([config.root_dir]) #it creates a directory at the root_dir path defined in config.yaml 

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [8]:
#update components
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)


In [10]:
#create pipeline

try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-29 18:23:01,028: INFO: common: yaml file: config\config.yaml loaded successfuly]
[2023-11-29 18:23:01,094: INFO: common: yaml file: params.yaml loaded successfuly]
[2023-11-29 18:23:01,109: INFO: common: Created directory at artifacts]
[2023-11-29 18:23:01,118: INFO: common: Created directory at artifacts/data_ingestion]
[2023-11-29 18:23:18,044: INFO: 3500835956: artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C2A6:1D35:D42C53:DD2052:65677376
Accept-Ranges: bytes
Date: Wed, 29 Nov 2023 17:23:03 GMT
Via: 1.1 varnish
X-Served-By: cache-mad2200125-MAD
X-Cache: MIS